# Tool execution notebook
 this notebook  is use to make the experiments before  coed in the CI  process

In [ ]:

# export R2R_API_KEY=...
import os
from dotenv import load_dotenv
from r2r import R2RClient


load_dotenv()
# https://r2r-docs.sciphi.ai/documentation/search-and-rag
def search(query: str):
  
  user = os.getenv("R2R_USER")  
  pwd = os.getenv("R2R_PWD")  
  base_url = os.getenv("R2R_URL", "http://localhost:7272")
  client = R2RClient(base_url=base_url) 
  client.users.login(user, pwd)

  response = client.retrieval.rag(
    query=query
  )
  return response


In [ ]:
## test the calls to search the documents
response = search("explain what is leadin and leadout in expert")


In [ ]:
print(response.results.generated_answer)

In [ ]:

print(response.results.search_results.to_json())

In [16]:
import os

from r2r import (
    R2RClient,
    ThinkingEvent,
    ToolCallEvent,
    ToolResultEvent,
    CitationEvent,
    FinalAnswerEvent,
    MessageEvent,
)

user = os.getenv("R2R_USER")
pwd = os.getenv("R2R_PWD")
base_url = os.getenv("R2R_URL", "http://localhost:7272")
client = R2RClient(base_url=base_url)
client.users.login(user, pwd)

# Basic synchronous request (already done, see 'response' variable)

# Advanced analysis with streaming and extended thinking
streaming_response = client.retrieval.agent(
    message={
        "role": "user",
        "content": "do a deep analisys of the new  updates in version 44 of expert"
    },
    search_settings={"limit": 20},
    rag_tools=[ "search_file_descriptions", "search_file_knowledge", "get_file_content"],
    rag_generation_config={
        "model": "openai/azure-gpt",
        "extended_thinking": True,
        "thinking_budget": 4096,
        "temperature": 1,
        "top_p": None,
        "max_tokens": 16000,
        "stream": True
    }
)

# Process streaming events with emoji only on type change
current_event_type = None
for event in streaming_response:
    event_type = type(event)
    if event_type != current_event_type:
        current_event_type = event_type
        print()  # Add newline before new event type

        # Print emoji based on the new event type
        if isinstance(event, ThinkingEvent):
            print("🧠 Thinking: ", end="", flush=True)
        elif isinstance(event, ToolCallEvent):
            print("🔧 Tool call: ", end="", flush=True)
        elif isinstance(event, ToolResultEvent):
            print("📊 Tool result: ", end="", flush=True)
        elif isinstance(event, CitationEvent):
            print("📑 Citation: ", end="", flush=True)
        elif isinstance(event, MessageEvent):
            print("💬 Message: ", end="", flush=True)
        elif isinstance(event, FinalAnswerEvent):
            print("✅ Final answer: ", end="", flush=True)

    # Print the content without the emoji
    if isinstance(event, ThinkingEvent):
        print(f"{event.data.delta.content[0].payload.value}", end="", flush=True)
    elif isinstance(event, ToolCallEvent):
        print(f"{event.data.name}({event.data.arguments})")
    elif isinstance(event, ToolResultEvent):
        print(f"{event.data.content[:60]}...")
    elif isinstance(event, CitationEvent):
        print(f"{event.data.id}")
    elif isinstance(event, MessageEvent):
        print(f"{event.data.delta.content[0].payload.value}", end="", flush=True)
    elif isinstance(event, FinalAnswerEvent):
        print(f"{event.data.generated_answer[:100]}...")
        print(f"   Citations: {len(event.data.citations)} sources referenced")




🔧 Tool call: search_file_knowledge({"query":"updates in version 44 of Lantek Expert Cut"})
get_file_content({"document_id": "3613b97c-a8db-5c9c-81b6-20bd4f807910"})
get_file_content({"document_id": "5c4f01e"})




In [19]:
# Conversation with multiple turns (synchronous)
conversation = client.conversations.create()

# First message in conversation
results_1 = client.retrieval.agent(
    message={"role": "user", "content": "what is leadin and leadout in expert"},
    rag_generation_config={
        "model": "openai/azure-gpt",
        "extended_thinking": True,
        "thinking_budget": 4096,
        "temperature": 1,
        "top_p": None,
        "max_tokens": 16000,
        "stream": False
    },
    conversation_id=conversation.results.id
)


# Access the final results
print(f"First response: {results_1}")


First response: results=AgentResponse(messages=[Message(role='assistant', content='The terms "lead-in" and "lead-out" in the context of Expert (likely referring to CAD/CAM or CNC machining software) pertain to the practice of setting up the approach and exit paths of cutting tools during the machining process. Here are some details about each:\n\n### Lead-In\n- **Definition**: A lead-in is the path that a cutting tool takes as it approaches the material to begin the cut. It is designed to minimize impact and help prepare the material for clean cutting.\n- **Functions**:\n  - **Automatic Lead-In**: The software can automatically introduce lead-ins in closed contours, improving the quality of the cut (e.g., for holes).\n  - **Parameters**: Users can define the length and angle of the lead-in to suit specific machining needs.\n  - **Considerations**: Various options exist for the lead-in configuration, such as specifying the lead-in\'s position and direction (e.g., normal to the surface, 

In [20]:
# Follow-up query in the same conversation
results_2 = client.retrieval.agent(
    message={"role": "user", "content": "i can I configure the leadin and leadout in expert"},
    rag_generation_config={
        "model": "openai/azure-gpt",
        "extended_thinking": True,
        "thinking_budget": 4096,
        "temperature": 1,
        "top_p": None,
        "max_tokens": 16000,
        "stream": False
    },
    conversation_id=conversation.results.id
)
print(f"Follow-up response: {results_2}...")

Follow-up response: results=AgentResponse(messages=[Message(role='assistant', content="Here's how you can configure lead-in and lead-out settings in the Expert Cut software, specifically focusing on various parameters and options available for your configuration:\n\n### Configuring Lead-In and Lead-Out in Expert Cut\n\n1. **Accessing the Configuration**\n   - Navigate to the **Machining** section where you need to configure lead-ins and lead-outs. \n   - Look for options related to **Automatic Machining** or **Cutting Profiles**.\n\n2. **Configuration Options**\n   - **Apply**: You will find an **Apply** option that allows you to add or modify lead-ins.\n   - **Closed Contours Settings**: Configure as follows:\n     - **Normal to Surface**: If selected, the cutting head will approach perpendicularly to the material surface.\n     - **Position Options**:\n       - **Corner Point:** Lead-in on the corner of the discredited point.\n       - **Middle Point:** Lead-in at the middle point of

In [ ]:
results_1